# 1.0 Working on Signature Methods

In [1]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)
df = {}

import clustergrammer_groupby as cby
import gene_exp_10x

In [2]:
### signature distance calculation requirements
from sklearn.metrics import pairwise_distances
from scipy.stats import ttest_ind 

import pandas as pd
import numpy as np
from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline 

### Load Default Dataset

In [3]:
net.load_file('../data/rc_two_cats.txt')
df['ini'] = net.export_df()
df['ini'].shape

### Drop Gender Category
cols = df['ini'].columns.tolist()
new_cols = []
for inst_col in cols:
    inst_col = list(inst_col)
    new_col = tuple(inst_col[:2])
    new_cols.append(new_col)

df['ini'].columns = new_cols

In [4]:
net.load_df(df['ini'])
net.cluster()
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.


### Calculate Signatures of Categories

In [5]:
df['ini_mi'] = cby.row_tuple_to_multiindex(df['ini'].transpose())

In [6]:
df['ini_mi'].head()

,,"(Gene: CDK4, Gene Type: Interesting)","(Gene: LMTK3, Gene Type: Not Interesting)","(Gene: LRRK2, Gene Type: Not Interesting)","(Gene: UHMK1, Gene Type: Not Interesting)","(Gene: EGFR, Gene Type: Interesting)","(Gene: STK32A, Gene Type: Interesting)","(Gene: NRK, Gene Type: Interesting)","(Gene: ERBB2, Gene Type: Not Interesting)","(Gene: ERBB4, Gene Type: Not Interesting)","(Gene: AAK1, Gene Type: Not Interesting)",...,"(Gene: ROS1, Gene Type: Interesting)","(Gene: MAP2K4, Gene Type: Interesting)","(Gene: SRC, Gene Type: Interesting)","(Gene: TGFBR1, Gene Type: Interesting)","(Gene: CAMK2B, Gene Type: Not Interesting)","(Gene: STK24, Gene Type: Interesting)","(Gene: DCLK3, Gene Type: Not Interesting)","(Gene: LATS1, Gene Type: Not Interesting)","(Gene: NEK9, Gene Type: Not Interesting)","(Gene: MYLK3, Gene Type: Not Interesting)"
Cell Line,Category,,,,,,,,,,,,,,,,,,,,,
H1650,two,-0.792804,0.177621,-0.697876,0.850547,1.412416,-0.388040,1.408537,0.906642,-0.452907,3.579052,...,-0.312364,0.119311,-0.294264,-0.000864,-0.276737,-0.318076,-0.670178,-0.695253,-0.337849,-0.368173
H23,two,0.527687,-0.016061,-0.555610,-0.263280,0.018988,-0.592627,-0.017369,-0.684771,-0.392791,0.923308,...,0.701257,0.593671,-0.618072,0.735638,-0.426081,-0.814111,3.224534,4.299877,-0.535266,0.209192
CAL-12T,two,0.000623,5.422114,-0.360498,0.179253,0.902252,-0.244137,-0.367128,0.015261,-0.374174,-0.651094,...,0.475208,0.489153,-0.252534,-0.680290,-0.160160,0.646545,0.145510,-0.175587,0.803160,0.266318
H358,one,0.356723,1.307040,-0.460237,0.398647,-0.178137,0.740365,0.313254,0.160568,-0.527418,0.952744,...,-0.585297,0.841683,-0.786607,0.040926,-0.890033,0.268372,0.107432,-0.061022,0.275911,-0.100657
H1975,two,0.933286,0.355815,-0.680761,1.537664,0.781819,3.023348,-0.162887,0.365002,-0.320103,-0.212733,...,-0.122694,1.064674,-0.228027,0.359330,-0.437405,-9.425121,-1.120493,-0.391646,0.981343,-0.336792


### Collect Top diff genes using T-test

In [7]:
num_top_genes = False
pval_cutoff = 0.05

cell_types = sorted(list(set(df['ini_mi'].index.get_level_values('Category').tolist())))

inst_level = 'Category'

keep_genes = []
keep_genes_dict = {}

for inst_ct in cell_types:
    
    inst_ct_mat = df['ini_mi'].xs(key=inst_ct, level=inst_level)
    inst_other_mat = df['ini_mi'].drop(inst_ct, level=inst_level)
    
    inst_stats, inst_pvals = ttest_ind(inst_ct_mat, inst_other_mat, axis=0)
    
    ser_pval = pd.Series(data=inst_pvals, index=df['ini_mi'].columns.tolist()).sort_values()

    if num_top_genes == False:
        ser_pval_keep = ser_pval[ser_pval < pval_cutoff]
    else:
        ser_pval_keep = ser_pval[:num_top_genes]

    print(ser_pval_keep)
    inst_keep = ser_pval_keep.index.tolist()
    keep_genes.extend(inst_keep)
    keep_genes_dict[inst_ct] = inst_keep
    
keep_genes = sorted(list(set(keep_genes)))

(Gene: NEK9, Gene Type: Not Interesting)    0.000724
(Gene: ULK4, Gene Type: Interesting)        0.002321
(Gene: ROS1, Gene Type: Interesting)        0.010923
(Gene: MAPK11, Gene Type: Interesting)      0.013899
dtype: float64
(Gene: MAPK4, Gene Type: Interesting)         0.008790
(Gene: TGFBR1, Gene Type: Interesting)        0.010232
(Gene: SRPK3, Gene Type: Not Interesting)     0.013685
(Gene: UHMK1, Gene Type: Not Interesting)     0.016737
(Gene: CAMK2B, Gene Type: Not Interesting)    0.025516
(Gene: GRK1, Gene Type: Not Interesting)      0.036758
dtype: float64
(Gene: STK32A, Gene Type: Interesting)     0.000172
(Gene: STK31, Gene Type: Interesting)      0.002281
(Gene: KDR, Gene Type: Not Interesting)    0.006222
(Gene: NRK, Gene Type: Interesting)        0.011511
(Gene: CDK4, Gene Type: Interesting)       0.045237
dtype: float64
(Gene: LRRK2, Gene Type: Not Interesting)    0.002408
dtype: float64
(Gene: PRKCE, Gene Type: Not Interesting)    0.005459
(Gene: STK39, Gene Type: Inter

### Visualize the Average Levels of the Informative Genes

In [8]:
df['ini_gbm'] = df['ini_mi'].groupby(level=inst_level).mean().transpose()
cols = df['ini_gbm'].columns.tolist()
new_cols = []
for inst_col in cols:
    new_col = (inst_col, inst_level + ': ' + inst_col)
    new_cols.append(new_col)
df['ini_gbm'].columns = new_cols

In [9]:
df['cat_sig'] = df['ini_gbm'].ix[keep_genes]

# Category Signature

In [10]:
net.load_df(df['cat_sig'])
net.cluster()
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.


### Visualize Informative Dimensions in Original Data
Generate a matrix with only category-informative dimensions/genes.

In [11]:
df['ini_info'] = df['ini'].ix[keep_genes]
df['ini_info'].shape

(19, 29)

In [12]:
net.load_df(df['ini_info'])
net.cluster()
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.


# Predict Category Based on Signature

In [27]:
df_data_ini.ix[keep_rows]

,"(Cell Line: H1650, Category: two)","(Cell Line: H23, Category: two)","(Cell Line: CAL-12T, Category: two)","(Cell Line: H358, Category: one)","(Cell Line: H1975, Category: two)","(Cell Line: HCC15, Category: two)","(Cell Line: H1355, Category: three)","(Cell Line: HCC827, Category: one)","(Cell Line: H2405, Category: five)","(Cell Line: HCC78, Category: five)",...,"(Cell Line: H460, Category: three)","(Cell Line: H2106, Category: four)","(Cell Line: H441, Category: one)","(Cell Line: H1944, Category: three)","(Cell Line: H1437, Category: four)","(Cell Line: H1734, Category: one)","(Cell Line: LOU-NH91, Category: five)","(Cell Line: HCC44, Category: four)","(Cell Line: A549, Category: four)","(Cell Line: H1781, Category: one)"
"(Gene: CAMK2B, Gene Type: Not Interesting)",-0.276737,-0.426081,-0.160160,-0.890033,-0.437405,0.143897,-0.573426,-0.486419,-0.536963,-0.657041,...,-0.431222,5.339203,-0.493779,-0.645000,-0.477985,-0.401580,-0.621782,-0.249395,-0.303365,-0.922343
"(Gene: CDK4, Gene Type: Interesting)",-0.792804,0.527687,0.000623,0.356723,0.933286,-0.131729,0.808452,4.240885,-0.540231,-0.981457,...,0.342635,-0.430912,-0.405906,0.199564,-1.122536,2.210335,0.405126,-0.089763,0.405126,0.340013
"(Gene: GRK1, Gene Type: Not Interesting)",-0.751527,0.497623,-0.142535,-0.882124,-1.151283,2.307907,-0.120321,-0.351270,-1.526179,-0.753268,...,-0.404834,0.736930,-0.944275,-0.078919,0.661648,-0.244949,3.051535,-0.107365,0.367536,-1.517986
"(Gene: KDR, Gene Type: Not Interesting)",-0.524310,-0.285995,-0.484872,0.176655,-0.139712,-0.352978,-0.192530,-0.601258,-0.427323,0.459404,...,-0.463128,-0.600408,4.430956,-0.374773,-0.206853,1.756105,0.832887,-0.314392,-0.313737,0.493420
"(Gene: LRRK2, Gene Type: Not Interesting)",-0.697876,-0.555610,-0.360498,-0.460237,-0.680761,-0.169464,1.715709,-0.517105,0.184988,0.810660,...,-0.377854,-0.713473,-0.377609,4.308905,-0.638132,-0.556114,-0.318146,-0.489583,1.677377,-0.682790
"(Gene: MAPK11, Gene Type: Interesting)",1.760301,-0.912260,-1.163346,-0.965892,-0.795153,-0.616300,-1.360744,-1.448292,-0.024089,-1.188869,...,-0.864064,-0.025445,1.890946,0.667806,-0.625764,-1.063340,3.222817,-0.001360,-0.203662,0.187670
"(Gene: MAPK4, Gene Type: Interesting)",0.184462,-0.526038,0.432087,-0.882312,0.246356,0.858755,0.052858,-1.118341,-0.846949,-0.778824,...,-0.221064,-0.861484,0.125416,-0.191849,2.374460,-0.741421,0.765439,1.029797,0.033079,0.440666
"(Gene: NEK9, Gene Type: Not Interesting)",-0.337849,-0.535266,0.803160,0.275911,0.981343,-0.748451,-0.092431,-0.326477,-0.381244,-0.575344,...,-0.545183,0.475562,0.016688,-0.172178,0.585187,-0.404800,-3.997318,0.711030,-0.470884,0.354386
"(Gene: NRK, Gene Type: Interesting)",1.408537,-0.017369,-0.367128,0.313254,-0.162887,0.027412,-0.281352,5.813846,-0.161707,0.472387,...,-0.222725,0.368499,1.493032,-0.341747,-0.343420,5.284809,-0.358157,-0.222932,-0.401391,-0.412479
"(Gene: PRKCE, Gene Type: Not Interesting)",0.006532,0.564827,3.695319,0.316255,-0.268738,0.936462,0.229152,0.649579,-0.330104,-0.504535,...,-0.710531,-1.334690,0.158108,-0.347820,-0.074497,-0.970508,-0.264794,-0.298649,-0.100909,-0.117421


In [24]:
keep_rows

[('Gene: CAMK2B', 'Gene Type: Not Interesting'),
 ('Gene: CDK4', 'Gene Type: Interesting'),
 ('Gene: GRK1', 'Gene Type: Not Interesting'),
 ('Gene: KDR', 'Gene Type: Not Interesting'),
 ('Gene: LRRK2', 'Gene Type: Not Interesting'),
 ('Gene: MAPK11', 'Gene Type: Interesting'),
 ('Gene: MAPK4', 'Gene Type: Interesting'),
 ('Gene: NEK9', 'Gene Type: Not Interesting'),
 ('Gene: NRK', 'Gene Type: Interesting'),
 ('Gene: PRKCE', 'Gene Type: Not Interesting'),
 ('Gene: ROS1', 'Gene Type: Interesting'),
 ('Gene: SRPK3', 'Gene Type: Not Interesting'),
 ('Gene: STK24', 'Gene Type: Interesting'),
 ('Gene: STK31', 'Gene Type: Interesting'),
 ('Gene: STK32A', 'Gene Type: Interesting'),
 ('Gene: STK39', 'Gene Type: Interesting'),
 ('Gene: TGFBR1', 'Gene Type: Interesting'),
 ('Gene: UHMK1', 'Gene Type: Not Interesting'),
 ('Gene: ULK4', 'Gene Type: Interesting')]

### 

In [39]:
dist_type = 'cosine'
df_sig = deepcopy(df['cat_sig'])
print('df_sig: ', df_sig.shape)
df_data_ini = deepcopy(df['ini'])

print('df_data_ini: ', df_data_ini.shape)

keep_rows = df_sig.index.tolist()
df_data = deepcopy(df_data_ini.ix[keep_rows])
print('df_data: ', df_data.shape)

df_sig:  (19, 5)
df_data_ini:  (38, 29)
df_data:  (19, 29)


In [48]:
df_data.shape

(19, 29)

In [50]:
df_sig.shape

(19, 5)

In [82]:
# # calculate cell type similarity matrix
# ct_sim_mat = 1 - pairwise_distances(df_sig, df_sig, metric=dist_type)
# rows = df_sig.index.tolist()
# df_ct_sim = pd.DataFrame(data=ct_sim_mat, index=rows, columns=rows)    


# calculate sim_mat of df_data and df_sig
cell_types = df_sig.columns.tolist()
barcodes = df_data.columns.tolist()
sim_mat = 1 - pairwise_distances(df_sig.transpose(), df_data.transpose(), metric=dist_type)
df_sim = pd.DataFrame(data=sim_mat, index=cell_types, columns=barcodes).transpose()
# print(df_sim.shape)

In [83]:
df_sim

,"(five, Category: five)","(four, Category: four)","(one, Category: one)","(three, Category: three)","(two, Category: two)"
"(Cell Line: H1650, Category: two)",0.228958,-0.148606,0.075138,-0.132418,0.216696
"(Cell Line: H23, Category: two)",0.194221,0.088920,-0.131172,0.210714,0.451656
"(Cell Line: CAL-12T, Category: two)",-0.229308,-0.207210,-0.062494,0.112294,0.434564
"(Cell Line: H358, Category: one)",-0.157056,-0.406595,0.636839,0.159597,0.066026
"(Cell Line: H1975, Category: two)",0.151154,0.167911,0.215330,0.427982,0.797531
"(Cell Line: HCC15, Category: two)",-0.187265,0.163717,-0.179106,0.182469,0.519587
"(Cell Line: H1355, Category: three)",0.018588,-0.215515,0.001759,0.861429,0.589752
"(Cell Line: HCC827, Category: one)",0.058493,0.117446,0.768012,0.141530,0.178826
"(Cell Line: H2405, Category: five)",0.646727,-0.488212,0.177732,-0.052729,-0.229075
"(Cell Line: HCC78, Category: five)",0.598129,-0.027581,0.256200,-0.066704,-0.044891


In [28]:




# # calculate sim_mat of df_data and df_sig
# cell_types = df_sig.index.tolist()
# barcodes = df_data.index.tolist()
# sim_mat = 1 - pairwise_distances(df_sig, df_data, metric=dist_type)
# df_sim = pd.DataFrame(data=sim_mat, index=cell_types, columns=barcodes).transpose() 
# print(df_sim.shape)

# ct_sim_mat = 1 - pairwise_distances(df_sim, df_ct_sim, metric=dist_type)
# df_ct_sim = pd.DataFrame(data=ct_sim_mat, index=barcodes, columns=cell_types)
# print(df_ct_sim.shape)

# num_match = 0
# num_mismatch = 0

# ser_list = []
# top_list = []
# rows = df_sim.index.tolist()
# for inst_row in rows:

#     # make ser_data_sim
#     inst_ser = df_sim.loc[[inst_row]]
#     inst_data = inst_ser.get_values()[0]
#     inst_cols = inst_ser.columns.tolist()
#     ser_data_sim = pd.Series(data=inst_data, index=inst_cols, name=inst_row).sort_values(ascending=False)

#     # define top matching cell type
#     top_ct_1 = ser_data_sim.index.tolist()[0]        

#     if top_ct_1 != top_ct_2:
#         num_mismatch = num_mismatch + 1
#     else:
#         num_match = num_match + 1

#     # use cell type signature
#     found_ct = top_ct_1

#     # calculate mean marker level (used to help assign doublets)
#     marker_mean = df_data.loc[[inst_row]].mean().mean()

#     # make binary matrix of ct_max
#     inst_zeros = np.zeros((len(inst_cols)))
#     max_ser = pd.Series(data=inst_zeros, index=inst_cols, name=inst_row)
#     max_ser[found_ct] = 1
#     top_list.append(found_ct)
#     ser_list.append(max_ser)

# # make matrix of top cell type identified 
# df_sim_top = pd.concat(ser_list, axis=1).transpose()

# # add cell type category to input data
# df_cat = deepcopy(df_data)
# rows = df_cat.index.tolist()
# new_rows = []
# for i in range(len(rows)):
#     inst_row = list(rows[i])
#     inst_row.append('Predict Cell Type: ' + top_list[i])
#     inst_tuple = tuple(inst_row)
#     new_rows.append(inst_tuple)
# df_cat.index = new_rows

# print('match: ', num_match)
# print('mismatch: ' , num_mismatch)


In [15]:
def assign_cell_type_from_sig_sim(df_data_ini, df_sig, dist_type='cosine', doublet_marker_high = 0.75,
                                  which_sig='cell-type'):
    
    keep_rows = df_sig.columns.tolist()
    df_data = deepcopy(df_data_ini[keep_rows])
    
    # calculate cell type similarity matrix
    ct_sim_mat = 1 - pairwise_distances(df_sig, df_sig, metric=dist_type)
    rows = df_sig.index.tolist()
    df_ct_sim = pd.DataFrame(data=ct_sim_mat, index=rows, columns=rows)    
    
    # calculate sim_mat of df_data and df_sig
    cell_types = df_sig.index.tolist()
    barcodes = df_data.index.tolist()
    sim_mat = 1 - pairwise_distances(df_sig, df_data, metric=dist_type)
    df_sim = pd.DataFrame(data=sim_mat, index=cell_types, columns=barcodes).transpose() 
    print(df_sim.shape)
    
    ct_sim_mat = 1 - pairwise_distances(df_sim, df_ct_sim, metric=dist_type)
    df_ct_sim = pd.DataFrame(data=ct_sim_mat, index=barcodes, columns=cell_types)
    print(df_ct_sim.shape)
    
    num_match = 0
    num_mismatch = 0
    
    ser_list = []
    top_list = []
    rows = df_sim.index.tolist()
    for inst_row in rows:

        # make ser_data_sim
        inst_ser = df_sim.loc[[inst_row]]
        inst_data = inst_ser.get_values()[0]
        inst_cols = inst_ser.columns.tolist()
        ser_data_sim = pd.Series(data=inst_data, index=inst_cols, name=inst_row).sort_values(ascending=False)
        
        # define top matching cell type
        top_ct_1 = ser_data_sim.index.tolist()[0]        
        
        if top_ct_1 != top_ct_2:
            num_mismatch = num_mismatch + 1
        else:
            num_match = num_match + 1
        
        # use cell type signature
        found_ct = top_ct_1

        # calculate mean marker level (used to help assign doublets)
        marker_mean = df_data.loc[[inst_row]].mean().mean()

        # make binary matrix of ct_max
        inst_zeros = np.zeros((len(inst_cols)))
        max_ser = pd.Series(data=inst_zeros, index=inst_cols, name=inst_row)
        max_ser[found_ct] = 1
        top_list.append(found_ct)
        ser_list.append(max_ser)

    # make matrix of top cell type identified 
    df_sim_top = pd.concat(ser_list, axis=1).transpose()
    
    # add cell type category to input data
    df_cat = deepcopy(df_data)
    rows = df_cat.index.tolist()
    new_rows = []
    for i in range(len(rows)):
        inst_row = list(rows[i])
        inst_row.append('Predict Cell Type: ' + top_list[i])
        inst_tuple = tuple(inst_row)
        new_rows.append(inst_tuple)
    df_cat.index = new_rows
    
    print('match: ', num_match)
    print('mismatch: ' , num_mismatch)
    
    return df_cat, df_sim, df_sim_top

In [18]:
df['ge_ct_pred'], df['ct_sim'], df['ct_max'] = assign_cell_type_from_sig_sim(df['ini'].transpose(), df['cat_sig'], 
                                                                              dist_type='cosine', which_sig='marker')

KeyError: "[('five', 'Category: five') ('four', 'Category: four')\n ('one', 'Category: one') ('three', 'Category: three')\n ('two', 'Category: two')] not in index"